In [1]:
from xbbg import blp
import pandas as pd


SAVEFILE = 'gmo_data.xlsx'

# Define tickers and fields for Bloomberg queries
tickers = [
    "SPX Index",          
    "USGG10YR Index",     
    "SPY US Equity",      
    "GMWAX US Equity",    
    "GMGEX US Equity",    
    "USGG3M Index"        
]

fields = {
    "SPX Index": ["EQY_DVD_YLD_12M", "PE_RATIO"],  
    "USGG10YR Index": ["PX_LAST"],         # 10-year Treasury yield
    "SPY US Equity": ["TOT_RETURN_INDEX_GROSS_DVDS"],  
    "GMWAX US Equity": ["TOT_RETURN_INDEX_GROSS_DVDS"],  
    "GMGEX US Equity": ["TOT_RETURN_INDEX_GROSS_DVDS"],  
    "USGG3M Index": ["PX_LAST"]            
}

# Set time range for monthly data
start_date = "1993-01-01"
end_date = "2024-10-31"

# Create a dictionary to store the results
data_dict = {}

# Fetch the time series data for each ticker and field
for ticker, field_list in fields.items():
    for field in field_list:
        # Query Bloomberg API using xbbg
        print(f"Fetching {ticker} - {field}...")
        data = blp.bdh(tickers=ticker, flds=field, start_date=start_date, end_date=end_date, Per="M")
        if data is not None and not data.empty:
            data_dict[f"{ticker} {field}"] = data
        else:
            print(f"Warning: No data returned for {ticker} - {field}")

# Filter out empty DataFrames
valid_data = {key: df for key, df in data_dict.items() if not df.empty}

# Combine all valid data into a single DataFrame
if valid_data:
    data_combined = pd.concat(valid_data.values(), axis=1)
    
    # Flatten MultiIndex columns if they exist and extract only the ticker name
    if isinstance(data_combined.columns, pd.MultiIndex):
        data_combined.columns = [' '.join(col).strip().split(' ')[0] for col in data_combined.columns]
    else:
        data_combined.columns = [col.split(' ')[0] for col in data_combined.columns]

    # Print columns for debugging
    print("Columns after processing:", data_combined.columns)

    # Drop rows containing any NaN values from data_combined
    data_combined.dropna(inplace=True)
    data_combined.index.name = 'date'

    # Prepare data for each sheet
    signals = data_combined[["SPX", "USGG10YR"]]
    signals.columns = ['SPX DVD YLD','SPX P/E','TNote 10YR']

    risk_free_rate = data_combined[["USGG3M"]] / 100
    risk_free_rate.rename(columns={'USGG3M':'TBill 3M'},inplace=True)

    total_returns = data_combined[["SPY", "GMWAX", "GMGEX"]].pct_change().dropna()

    # Intersect the dates
    idx_common = signals.index.intersection(risk_free_rate.index).intersection(total_returns.index)
    signals = signals.loc[idx_common]
    risk_free_rate = risk_free_rate.loc[idx_common]
    total_returns = total_returns.loc[idx_common]

    # Create a summary info sheet
    info = pd.DataFrame({
        "Ticker": (signals.columns.to_list() + risk_free_rate.columns.to_list() + total_returns.columns.to_list()),
        "Description": [
            "S&P 500 Index for dividend yield",
            "S&P 500 Index for price-earning multiple",
            "10-year Treasury yield",
            "3-month T-Bill rate",
            "SPY ETF for returns and dividend yield",
            "GMWAX mutual fund for returns",
            "GMO Global Equity Allocation Fund for returns"
        ]
    })

    # Export to Excel with multiple sheets
    with pd.ExcelWriter(SAVEFILE) as writer:
        info.to_excel(writer, sheet_name="info", index=False)
        signals.to_excel(writer, sheet_name="signals")
        risk_free_rate.to_excel(writer, sheet_name="risk-free rate")
        total_returns.to_excel(writer, sheet_name="total returns")

    print("Data successfully exported.")
else:
    print("No valid data was fetched.")


Fetching SPX Index - EQY_DVD_YLD_12M...
Fetching SPX Index - PE_RATIO...
Fetching USGG10YR Index - PX_LAST...
Fetching SPY US Equity - TOT_RETURN_INDEX_GROSS_DVDS...
Fetching GMWAX US Equity - TOT_RETURN_INDEX_GROSS_DVDS...
Fetching GMGEX US Equity - TOT_RETURN_INDEX_GROSS_DVDS...
Fetching USGG3M Index - PX_LAST...
Columns after processing: Index(['SPX', 'SPX', 'USGG10YR', 'SPY', 'GMWAX', 'GMGEX', 'USGG3M'], dtype='object')
Data successfully exported.
